In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import crocoddyl
from utils import *
from neural_net import *
from outliers import *
import time
import matplotlib.pyplot as plt



In [3]:
# Load the two neural nets

# net1 is the neural net trained on 1K datapoints
net1 = torch.load("net1.pth")

# net2 is the neural net trained on 100K points
#net2 = torch.load("net2.pth") 

In [4]:
class UnicycleTerminal(crocoddyl.ActionModelAbstract):
    """
    This class is the terminal model for unicycle crocoddyl with neural net inside it.
    
    Given a state x, the neural net predicts the value function in the calc.
            # net(x) = data.cost
            
    Given the state x, the jacobian and hessian of the net(x) with respect to x are calculated
            # jacobian(net(x), x) = data.Lx
            # hessian(net(x), x)  = data.Lxx
    
    """
    def __init__(self, net):
        """
        @params
            1: network
        
        usage: terminal_model = UnicycleTerminal(net)   
        
        """
        crocoddyl.ActionModelAbstract.__init__(self, crocoddyl.StateVector(3), 2, 5)
        self.net = net
        
        
    def calc(self, data, x, u=None):
        
        x = torch.as_tensor(m2a(x), dtype = torch.float32).resize_(1, 3)
        
        with torch.no_grad():
            data.cost = self.net(x).item()

    def calcDiff(self, data, x, u=None):
        
        if u is None:
            u = self.unone
        
        x0 = torch.as_tensor(m2a(x), dtype = torch.float32).resize_(1, 3)
        
        x0.requires_grad_(True)
        
        j = jacobian(self.net(x0), x0)        
        h = hessian(self.net(x0), x0)
        
        data.Lx = a2m(j.detach().numpy())
        data.Lxx = a2m(h.detach().numpy())
        
        _h = h.detach().numpy().reshape(3, 3)
        #print(f"Hessian {_h}")
        #print(f"EigenVals of Hessian are {np.linalg.eigvals(_h)}")
        #print("...")
        

        

In [ ]:
plot_neural_net(net1, "net1")

In [ ]:
plot_2(torchnet, "net1_x_cons")